In [3]:
# 검증 세트
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [4]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [5]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [6]:
# 훈련 세트를 다시 train_test_split 함수에 넣어 검증 세트 만들기
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)

In [7]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [10]:
# 검증 세트로 모델 생성 및 평가
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [11]:
# 교차 검증
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00627685, 0.00523973, 0.00501513, 0.00458074, 0.00406671]), 'score_time': array([0.000705  , 0.00111318, 0.00043082, 0.00067616, 0.00031018]), 'test_score': array([0.87019231, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [12]:
# 교차검증의 최종점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있음
import numpy as np
print(np.mean(scores['test_score']))

0.8554925223957948


In [13]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.8554925223957948


In [15]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) # 매개변수 10으로 설정
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8581873425226026


In [17]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [18]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params,n_jobs=-1)

In [19]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [21]:
# 그리드 서치는 훈련이 끝나면 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련함
dt = gs.best_estimator_ # gs객체의 best_estimator에 저장됨
print(dt.score(train_input, train_target))

0.9615162593804117


In [22]:
# 그리드 서치로 찾은 최적의 매개변수는 best_params에 저장됨
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [23]:
# 각 매개변수에서 수행한 교차 검증의 평균점수는 cv_results_ 속성의 mean_test_score키에 저장됨
print(gs.cv_results_['mean_test_score'])

[0.86800067 0.86453617 0.86492226 0.86780891 0.86761605]


In [24]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


- 1. 먼저 탐색할 매개변수를 지정
- 2. 그 다음 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾음. 이 조합은 그리드 서치 객체에 저장됨  
- 3. 그리드 서치는 최상의 매개변수에서 (교차 검증에 사용한 훈련 세트가 아니라) 전체 훈련 세트를 사용해 최종 모델을 훈련함. 이 모델은 그리드 서치 객체에 저장됨

In [26]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001), # arange는 첫 번째 매개변수 값에서 시작하여 두번쨰 매개변수에 도달할 때까지 세번째 매개변수를 계속 더한 배열을 만듭니다 0.0001~ 0.001이 될까지 계속 더한 배열
          'max_depth' : range(5,20,1), # 트리의 깊이를 제한하고 5에서 20까지 1씩 증가하면서 15개의 값을 만듭니다.
          'min_samples_split' : range(2,100,10)}  # 노드를 나누기 위한 최소 샘플 수. 2에서 100까지 10씩 증가하면서 10개의 값을 만듭니다.

In [27]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [28]:
# 최상의 매개변수 조합 확인
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [29]:
# 최상의 교차 검증 점수
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [31]:
# 랜덤 서치
from scipy.stats import uniform, randint
rgen = randint(0,10)
rgen.rvs(10)

array([6, 2, 2, 3, 5, 5, 9, 4, 4, 1])

In [32]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([103,  97,  96, 100,  92, 101, 108, 119,  85,  99]))

In [33]:
ugen = uniform

In [34]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.23487259, 0.41975904, 0.35583311, 0.99705545, 0.85638691,
       0.85611694, 0.76653272, 0.53711531, 0.23680758, 0.23233638])

In [35]:
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth' : randint(20, 50),
          'min_samples_split' : randint(2, 25),
          'min_samples_leaf' : randint(1, 25),}

In [36]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x11ed52140>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x12bd7db10>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x12e28baf0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x12e28b0a0>},
                   random_state=42)

In [40]:
# 최적의 매개변수 조합
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [41]:
# 최고 교차 검증 점수
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [42]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target)) # 테스트 세트의 성능 확인

0.86
